In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.nn import functional as F

In [3]:
#gets training and testing data for MNIST dataset
trainData = datasets.MNIST('data/', train = True, transform = transforms.ToTensor(), download=True)
testData = datasets.MNIST('data/', train = False, transform = transforms.ToTensor(), download=True)

#constructs loaders from datasets
trainLoader = torch.utils.data.DataLoader(trainData, batch_size=100, shuffle=True)
testLoader = torch.utils.data.DataLoader(testData, batch_size=100, shuffle=True)

In [24]:
'''
Convolutional layers are instantiated with Conv2d:
self.newLayer = nn.Conv2d(input_channels, output_channels, filter_size)
'''

'''
First Convolutional Neural Network
'''
class ConvNet1(nn.Module):
    def __init__(self):
        super(ConvNet1, self).__init__()
        #(1,28,28) -> (8,22,22)
        self.conv1 = nn.Conv2d(1, 8, 7)
        
        #(8,22,22) -> (16,17,17)
        self.conv2 = nn.Conv2d(8, 16, 6)
        
        #(16,17,17) -> (32,12,12)
        self.conv3 = nn.Conv2d(16, 32, 6)
        
        #(32,12,12) -> (64,8,8)
        self.conv4 = nn.Conv2d(32, 64, 5)

        #(64,8,8) -> (80,4,4)
        self.conv5 = nn.Conv2d(64, 80, 5)
        
        #(80,4,4) -> (100,2,2)
        self.conv6 = nn.Conv2d(80, 100, 3)
        
        #400 -> 10
        self.linear = nn.Linear(400, 10)
        
    def forward(self, x):
        #(1,28,28) -> (8,22,22)
        x = torch.sigmoid(self.conv1(x))
        
        #(8,22,22) -> (16,17,17)
        x = torch.sigmoid(self.conv2(x))
        
        #(16,17,17) -> (32,12,12)
        x = torch.sigmoid(self.conv3(x))
        
        #(32,12,12) -> (64,8,8)
        x = torch.sigmoid(self.conv4(x))
        
        #(64,8,8) -> (80,4,4)
        x = torch.sigmoid(self.conv5(x))
        
        #(80,4,4) -> (100,2,2)
        x = torch.sigmoid(self.conv6(x))
        
        #(100,2,2) -> 400
        x = x.view(-1, 400)
        
        #400 -> 10
        x = F.softmax(self.linear(x), dim=1)
        
        return x        
    
model = ConvNet1()
optimizer = torch.optim.Adam(model.parameters())

In [14]:
def train(epoch):
    trainingLoss = 0
    for index, (data, target) in enumerate(trainLoader):
        optimizer.zero_grad()
        predictions = model(data)
        batchLoss = F.cross_entropy(predictions, target)
        batchLoss.backward()
        trainingLoss += batchLoss.item()
        optimizer.step()
        if index % 10 == 0:
            print(f'Training Epoch: {epoch} [{index * len(data)} / {len(trainData)}]\tLoss: {(batchLoss.item() / len(data)):.6f}')
    print(f'----- Epoch: {epoch} Average loss: {(trainingLoss/len(trainData)):.4f}')
    
def test():
    testingLoss = 0
    total = 0
    correct = 0
    with torch.no_grad():
        for index, (data, target) in enumerate(testLoader):
            predictions = model(data)
            testingLoss += F.cross_entropy(predictions, target).item()
            _, predictedValues = torch.max(predictions, 1)
            print('target is: ')
            print(target)
            print('predicted values are: ')
            print(predictedValues)
            total += target.size(0)
            correct += (predictedValues == target).sum().item()
    print(f'----- Test set loss: {(testingLoss):.4f}')
    print(f'----- Accuracy: {correct/total}')

In [15]:
for epoch in range(1,11):
    train(epoch)
    test()

Training Epoch: 1 [0 / 60000]	Loss: 0.023072
Training Epoch: 1 [1000 / 60000]	Loss: 0.023036
Training Epoch: 1 [2000 / 60000]	Loss: 0.022984
Training Epoch: 1 [3000 / 60000]	Loss: 0.023044
Training Epoch: 1 [4000 / 60000]	Loss: 0.023067
Training Epoch: 1 [5000 / 60000]	Loss: 0.022969
Training Epoch: 1 [6000 / 60000]	Loss: 0.023102
Training Epoch: 1 [7000 / 60000]	Loss: 0.022996
Training Epoch: 1 [8000 / 60000]	Loss: 0.023058
Training Epoch: 1 [9000 / 60000]	Loss: 0.022964
Training Epoch: 1 [10000 / 60000]	Loss: 0.023067
Training Epoch: 1 [11000 / 60000]	Loss: 0.022962
Training Epoch: 1 [12000 / 60000]	Loss: 0.023026
Training Epoch: 1 [13000 / 60000]	Loss: 0.022774
Training Epoch: 1 [14000 / 60000]	Loss: 0.022570
Training Epoch: 1 [15000 / 60000]	Loss: 0.022644
Training Epoch: 1 [16000 / 60000]	Loss: 0.022271
Training Epoch: 1 [17000 / 60000]	Loss: 0.022219
Training Epoch: 1 [18000 / 60000]	Loss: 0.022322
Training Epoch: 1 [19000 / 60000]	Loss: 0.022071
Training Epoch: 1 [20000 / 60000]

KeyboardInterrupt: 

In [47]:
'''
Second Convolutional Neural Network
'''
class ConvNet2(nn.Module):
    def __init__(self):
        super(ConvNet2, self).__init__()
        #(1,28,28) -> (16,24,22)
        self.conv1 = nn.Conv2d(1, 16, (5,7))
        
        #(16,24,22) -> (32,21,19)
        self.conv2 = nn.Conv2d(16, 32, 8, padding=2)
        
        #(32,21,19) -> (64,10,9)
        self.conv3 = nn.Conv2d(32, 64, 3, stride=2)
        
        #(64,10,9) -> (128,6,6)
        self.conv4 = nn.Conv2d(64, 128, (5,6), padding=(0,1))
        
        #(128,6,6) -> (256,2,2)
        self.conv5 = nn.Conv2d(128, 256, 5)
        
        #1024 -> 10
        self.linear = nn.Linear(1024, 10)
        
    def forward(self, x):
        #(1,28,28) -> (16,24,22)
        x = torch.sigmoid(self.conv1(x))  
        
        #(16,24,22) -> (32,21,19)
        x = torch.sigmoid(self.conv2(x))  
        
        #(32,21,19) -> (64,10,9)
        x = torch.sigmoid(self.conv3(x))
        
        #(64,10,9) -> (128,6,6)
        x = torch.sigmoid(self.conv4(x))
        
        #(128,6,6) -> (256,2,2)
        x = torch.sigmoid(self.conv5(x))
        
        #(256,2,2) -> 1024
        x = x.view(-1, 1024)
        
        #1024 -> 10
        x = F.softmax(self.linear(x), dim=1)
        
        return x        
    
model = ConvNet2()
optimizer = torch.optim.Adam(model.parameters())

In [13]:
'''
Exercise: Make a Convolutional Neural Network meeting the following specifications:
1. Apply a convolutional layer with a padding and stride of 1 that takes in one input channel
and produces 14 output channels of size 22 x 25.
2. Apply a convolutional layer with a kernel size of 5 x 9 that produces 20 output channels of size 8 x 7
3. Apply a convolutional layer with a kernel size of 5 x 7 that produces 25 output channels of size 8 x 7
4. Apply any number of convolutional layers to yield an output of 30 channels of size 4 x 4
5. Flatten the output
6. Apply two linear layers to transform the output to a vector of length 10. Apply softmax and return.
'''
class ConvExercise(nn.Module):
    def __init__(self):
        super(ConvExercise, self).__init__()
        
        #(1,28,28) -> (12, 22, 25)
        self.conv1 = nn.Conv2d(1, 12, (9,6), padding=1)
        
        #(12, 22, 25) -> (20, 8, 7)
        self.conv2 = nn.Conv2d(12, 20, (5,9), stride=3, padding=(2,1))
        
        #(20, 8, 7) -> (25, 8, 7)
        self.conv3 = nn.Conv2d(20, 25, (5,7), padding=(2,3))
        
        #(25, 8, 7) -> (30, 4, 4)
        self.conv4 = nn.Conv2d(25, 30, (5,4))
        
        #480 -> 128
        self.linear1 = nn.Linear(480, 128)
        
        #128 -> 10
        self.linear2 = nn.Linear(128, 10)
        
    def forward(self, x):
        #(1,28,28) -> (12, 22, 25)
        x = torch.sigmoid(self.conv1(x))
        
        #(12, 22, 25) -> (20, 8, 7)
        x = torch.sigmoid(self.conv2(x))
        
        #(20, 8, 7) -> (25, 8, 7)
        x = torch.sigmoid(self.conv3(x))
        
        #(25, 8, 7) -> (30, 4, 4)
        x = torch.sigmoid(self.conv4(x))
        
        #(30, 4, 4) -> 480
        x = x.view(-1, 480)
        
        #480 -> 128
        x = torch.sigmoid(self.linear1(x))
        
        #128 -> 10
        x = F.softmax(self.linear2(x), dim=1)
        
        return x

model = ConvExercise()
optimizer = optim.Adam(model.parameters())

In [35]:
'''
Convolutional Neural Network with Max Pooling
'''
class ConvNet3(nn.Module):
    def __init__(self):
        super(ConvNet3, self).__init__()
        
        #(1,28,28) -> (16,24,24)
        self.conv1 = nn.Conv2d(1, 16, 5)
        
        #(16,12,12) -> (32,10,10)
        self.conv2 = nn.Conv2d(16, 32, 3)
        
        #(32,5,5) -> (64,3,3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        
        #(64,3,3) -> (64,3,3)
        self.dropout = nn.Dropout2d(.2)
        
        #(64,3,3) -> (128,2,2)
        self.conv4 = nn.Conv2d(64, 128, 2)
        
        #512 -> 10
        self.linear = nn.Linear(512, 10)
        
    def forward(self, x):
        #(1,28,28) -> (16,24,24)
        x = torch.sigmoid(self.conv1(x))
        
        #(16,24,24) -> (16,12,12)
        x = F.max_pool2d(x, (2,2))
        
        #(16,12,12) -> (32,10,10)
        x = torch.sigmoid(self.conv2(x))
        
        #(32,10,10) -> (32,5,5)
        x = F.max_pool2d(x, (2,2))
        
        #(32,5,5) -> (64,3,3)
        x = torch.sigmoid(self.conv3(x))
        
        #(64,3,3) -> (64,3,3)
        x = self.dropout(x)
        
        #(64,3,3) -> (128,2,2)
        x = torch.sigmoid(self.conv4(x))
        
        #(128,2,2) -> 512
        x = x.view(-1, 512)
        
        #512 -> 10
        x = F.softmax(self.linear(x), dim=1)
        
        return x        
    
model = ConvNet3()
optimizer = torch.optim.Adam(model.parameters())

In [9]:
'''
Convolutional Neural Network with Max Pooling and new activation functions
'''
class ConvNet4(nn.Module):
    def __init__(self):
        super(ConvNet4, self).__init__()
        #(1,28,28) -> (16,24,24)
        self.conv1 = nn.Conv2d(1, 16, 5)
        
        #(16,12,12) -> (32,10,10)
        self.conv2 = nn.Conv2d(16, 32, 3)
        
        #(32,5,5) -> (64,3,3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        
        #(64,3,3) -> (64,3,3)
        self.dropout = nn.Dropout2d(.2)
        
        #(64,3,3) -> (128,2,2)
        self.conv4 = nn.Conv2d(64, 128, 2)
        
        #(128,2,2) -> (128,2,2)
        self.prelu = nn.PReLU()
        
        #512 -> 10
        self.linear = nn.Linear(512, 10)
        
    def forward(self, x):
        #(1,28,28) -> (16,24,24)
        x = F.relu(self.conv1(x))
        
        #(16,24,24) -> (16,12,12)
        x = F.max_pool2d(x, (2,2))
        
        #(16,12,12) -> (32,10,10)
        x = F.leaky_relu(self.conv2(x))
        
        #(32,10,10) -> (32,5,5)
        x = F.max_pool2d(x, (2,2))
        
        #(32,5,5) -> (64,3,3)
        x = F.leaky_relu(self.conv3(x), .0001)
        
        #(64,3,3) -> (64,3,3)
        x = self.dropout(x)
        
        #(64,3,3) -> (128,2,2)
        x = self.prelu(self.conv4(x))
        
        #(128,2,2) -> 512
        x = x.view(-1, 512)
        
        #512 -> 10
        x = F.softmax(self.linear(x), dim=1)
        
        return x        
    
model = ConvNet4()
optimizer = torch.optim.Adam(model.parameters())